This notebook is used just in order to play around in Python and try different things.

In [23]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
# convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
!pip install matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library
!pip install folium
import folium

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

import xml.etree.ElementTree as et 

print("Done!")

Done


In [6]:
address = 'Stockholm, Sweden'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


Let's scrape a table from Wikipedia that contains information about the metro stations:

In [182]:
source=requests.get('https://sv.wikipedia.org/wiki/Lista_%C3%B6ver_tunnelbanestationer_i_Stockholm').text
soup = BeautifulSoup(source,'lxml')
table=soup.find('table',{'class':'sortable wikitable'})
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

df = pd.DataFrame(res, columns=["Station", "Line", "Year", "Distance", "Municipality", "Type", "Latitude", "Longitude"])

In [183]:
df.head(110)

,Station,Line,Year,Distance,Municipality,Type,Latitude,Longitude
0,Slussen,Grön,1950,"1,5",Stockholms kommun,Betongstation,18.072327,59.319493
1,Medborgarplatsen,Grön,1950,"2,1",Stockholms kommun,Betongstation,18.073550,59.314342
2,Skanstull,Grön,1950,"2,7",Stockholms kommun,Betongstation,18.076229,59.307852
3,Gullmarsplan,Grön,1950,"3,8",Stockholms kommun,"Ytstation, delvis överbyggd av betong",18.080768,59.299114
4,Skärmarbrink,Grön,1950,"4,5",Stockholms kommun,Ytstation,18.090440,59.295366
5,Blåsut,Grön,1950,"5,1",Stockholms kommun,Ytstation,18.091061,59.290242
6,Sandsborg,Grön,1950,"5,7",Stockholms kommun,Ytstation,18.092382,59.284785
7,Skogskyrkogården,Grön,1950,"6,5",Stockholms kommun,Ytstation,18.095501,59.279194
8,Tallkrogen,Grön,1950,"7,5",Stockholms kommun,Ytstation,18.085326,59.271140
9,Gubbängen,Grön,1950,"8,2",Stockholms kommun,Ytstation,18.082036,59.262879


Let's create a new dataframe where:
* Only opened stations are left 
* All columns that are not going to be used are skipped
* Dublicates are removed
* Information about previous names is removed since it's not relevant for this assigment

In [203]:
df1=df[df['Year'].astype(str).str.isdigit()]
df1
df2=df1[['Station','Latitude','Longitude']]
df2
df2.describe()

,Station,Latitude,Longitude
count,105,105,105
unique,100,100,100
top,T-Centralen,18.059266,59.330945
freq,3,3,3


In [205]:
df3=df2.drop_duplicates(["Station"]).reset_index(drop=True)
df3.describe()
df3

,Station,Latitude,Longitude
0,Slussen,18.072327,59.319493
1,Medborgarplatsen,18.073550,59.314342
2,Skanstull,18.076229,59.307852
3,Gullmarsplan,18.080768,59.299114
4,Skärmarbrink,18.090440,59.295366
5,Blåsut,18.091061,59.290242
6,Sandsborg,18.092382,59.284785
7,Skogskyrkogården,18.095501,59.279194
8,Tallkrogen,18.085326,59.271140
9,Gubbängen,18.082036,59.262879


Let's skip all the columns that we are not going to use in this assigment:

In [206]:
#df2=df1[['Station']]
#df2

Let's remove information about previous station names:

In [207]:
df4['Station'] = df3['Station'].str.replace(r" \(.*\)","")
df4

C:\Users\oxana_r7lkrla\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Station
0,Slussen
1,Medborgarplatsen
2,Skanstull
3,Gullmarsplan
4,Skärmarbrink
5,Blåsut
6,Sandsborg
7,Skogskyrkogården
8,Tallkrogen
9,Gubbängen


Now let's add word "T-bana" to make our search in the next step more specific and therefore avoid a lot of cleaning afterwards:

In [176]:
#df4=df3
#df4['Station']=df4['Station'] + ' T-bana'
#df4

In [177]:
#df7=df4.iloc[90:120, :]
#df7

Now let's send an API request to get the coordinates for every metro station:

In [178]:
#url='https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&input=Gamla%20stan%20T-bana&format=json'
#response=requests.get(url).json()
#response

Let's keep all of the columns to begin with in order to use this information to make some checks that the data quality is OK:

In [179]:
#df5=response['StopLocation']
#df6 = json_normalize(df5)
#df6.head()

Let's create a list of metro stations that will be used for API call:

In [180]:
#stations=list(df7["Station"])
#stations

In [181]:
#response['StopLocation'][0]['id']

In [172]:
#st_list = []

# For each metro station, we access its information through its API
#for Station in stations:
 #   JSONContent = requests.get("https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&format=json&input=" + Station).json()
  #  st_list.append([JSONContent['StopLocation'][0]['id'], JSONContent['StopLocation'][0]['extId'], JSONContent['StopLocation'][0]['name'],
   #                      JSONContent['StopLocation'][0]['lon'], JSONContent['StopLocation'][0]['lat'], JSONContent['StopLocation'][0]['weight'], JSONContent['StopLocation'][0]['products']])
                         
#dataset = pd.DataFrame(st_list)
#dataset

In [173]:
#dataset

In [1]:
# ruby grab_venues.rb will print a csv of all the foursquare venue types
# flattens the nested hash of venues

url = "https://api.foursquare.com/v2/venues/categories?oauth_token=your_token&v=20120828"

require 'json'

# hashes are nested, containing arrays of hashes of category information
def print_hash(hash)
  # We've found a category
  if hash['id'] and hash['name']
    puts "#{hash['id']},#{hash['name']}"
  end
  # We've found an array of hashes of categories (that might contain more arrays!)
  if hash["categories"]
    hash["categories"].each do |h|
      print_hash(h)
    end
  end
end

ret = `curl #{url}`
json_response = JSON.parse(ret)

puts json_response["response"]["categories"].first

json_response["response"]["categories"].each do |c|
  print_hash(c)
end

SyntaxError: invalid syntax (<ipython-input-1-e8a07b116e6f>, line 6)

In [175]:
#JSONContent

In [ ]:
#url='https://api.resrobot.se/v2/location.name?key=57784d15-1b13-49bd-9fea-57c5d71af270&input=Gamla%20stan%20T-bana'
#response=requests.get(url).
#print(response)

In [174]:
#df4['Station']

In [12]:
#map_sthlm = folium.Map(location=[latitude, longitude], zoom_start=10)
#map_sthlm

In [ ]:
CLIENT_ID = 'RWLY0RHFLLB2X5L5SF2FV243QOFFSINLN0HZUOT1BFEBCK5G' # your Foursquare ID
CLIENT_SECRET = 'SBFSJDUVKAH2EJRJFRDS1ZYIQYLTKOTXAYREQN5AENT4L5PA' # your Foursquare Secret
VERSION = '20190623' # Foursquare API version

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, postcode, borough, neighbourhood in zip(df3['Latitude'], df3['Longitude'], df3['Postcode'], df3['Borough'], df3['Neighbourhood']):
    label = '{}, {}, {}'.format(postcode, borough, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore boroughs that contain the word "Toronto"

Before I slice the data, I would like to know how many and which boroughs there are in the dataset and how many different postcodes there are in each of them:

In [7]:
df3.groupby('Borough')['Postcode'].nunique()

Borough
Central Toronto      9
Downtown Toronto    18
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Queen's Park         1
Scarborough         17
West Toronto         6
York                 5
Name: Postcode, dtype: int64

 Now I create a new dataframe (b_toronto) with only boroughs that contain the word Toronto:

In [8]:
b_toronto= df3 [df3['Borough'].str.contains("Toronto") == True].reset_index(drop=True)
b_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


I expect the dataset to have 9+18+5+6=38 rows and 5 columns:

In [9]:
b_toronto.shape

(38, 5)

Now I create a new map in order to visualize the new dataframe:

In [10]:
# create map of boroughs that include the word Toronto using latitude and longitude values
map_b_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(b_toronto['Latitude'], b_toronto['Longitude'], b_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_b_toronto)  
    
map_b_toronto

To define Foursquare credentials and version:

To create a function that gets venues for all the boroughs in b_toronto: 

To run the function above on each neighbourhood in the b_toronto dataset:

In [13]:
b_toronto_venues = getNearbyVenues(names=b_toronto['Neighbourhood'],
                                   latitudes=b_toronto['Latitude'],
                                   longitudes=b_toronto['Longitude']
                                  )


The Beaches


NameError: name 'LIMIT' is not defined

To check the size and first 20 rows of the resulting dataframe:

In [ ]:
b_toronto_venues.shape

In [ ]:
b_toronto_venues.head(20)

To check how many venues were returned for each neighbourhood:

In [ ]:
b_toronto_venues.groupby('Neighbourhood').count()

To find out how many unique categories can be curated from all the returned venues:

In [ ]:
print('There are {} uniques categories.'.format(len(b_toronto_venues['Venue Category'].unique())))

To analyze each neighbourhood:

In [ ]:
# one hot encoding
b_toronto_onehot = pd.get_dummies(b_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
b_toronto_onehot['Neighbourhood'] = b_toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [b_toronto_onehot.columns[-1]] + list(b_toronto_onehot.columns[:-1])
b_toronto_onehot = b_toronto_onehot[fixed_columns]

b_toronto_onehot.head()

To check the dataframe's size:

In [ ]:
b_toronto_onehot.shape

To group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category:

In [ ]:
b_toronto_grouped = b_toronto_onehot.groupby('Neighbourhood').mean().reset_index()
b_toronto_grouped.head()

The new size:

In [ ]:
b_toronto_grouped.shape

In [ ]:
b_toronto_grouped.head()

To return top 8 venues (just to try something else than 10) for each group of neighbourhoods:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = b_toronto_grouped['Neighbourhood']

for ind in np.arange(b_toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(b_toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(40)

### Cluster Neighbourhoods
I've tried to use different number of clusters and decided to keep the version with 5 clusters here:

In [ ]:
# set number of clusters
kclusters = 5

b_toronto_grouped_clustering = b_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(b_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

To create a new dataframe that includes the cluster as well as the top 10 venues for each neighbourhood:

In [ ]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

b_toronto_merged = b_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
b_toronto_merged = b_toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

b_toronto_merged.head()

To visualize the result:

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_toronto_merged['Latitude'], b_toronto_merged['Longitude'], b_toronto_merged['Neighbourhood'], b_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Next I'll examine each cluster and try to make some conclusions at the end of this notebook.

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 0, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 1, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 2, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 3, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

In [ ]:
b_toronto_merged.loc[b_toronto_merged['Cluster Labels'] == 4, b_toronto_merged.columns[[1] + list(range(5, b_toronto_merged.shape[1]))]]

### Some conclusions

The purpose of the exercise was to explore Toronto and some of its neighbourhoods, to try out clustering and to visualize the results.

The clustering itself would need to be improved in order to be able to describe the clusters in a meaningful way since 4 out of 5 clusters contain just one or two neighbourhoods while one of the clusters contains almost all the neighbourhoods. Since this kind of improvements are outside the scope of the task, I'll stop here.

Thank you for reviewing this notebook!